In [40]:
%matplotlib inline
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [2]:
# https://medium.com/jdsc-tech-blog/multioutput-cnn-in-pytorch-c5f702d4915f
##############################################################
# DATA LOADING
##############################################################

#########################
# HUMANS IMAGES
#########################


humans_path = 'pattern_ii_dataset/humans/dataset/synthetic_images'

all_humans = []
    
for root, dirs, files in sorted(os.walk(humans_path)):
    
    for name in files:
        all_humans.append(str(root) + "/" + str(name))
        
print("loaded %d files into human database" % len(all_humans))

#########################
# MONSTERS IMAGES
#########################

monsters_path = 'pattern_ii_dataset/monsters/dataset/synthetic_images'

all_monsters = []
    
for root, dirs, files in sorted(os.walk(monsters_path)):
    
    for name in files:
        all_monsters.append(str(root) + "/" + str(name))
        
print("loaded %d files into monster database" % len(all_monsters))

#########################
# HUMANS ANNOTATIONS
#########################

humans_annotations_path = 'pattern_ii_dataset/humans/dataset/annotations'

all_humans_annotations = []
    
for root, dirs, files in sorted(os.walk(humans_annotations_path)):
    
    for name in files:
        all_humans_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_humans_annotations database" % len(all_humans_annotations))

print(all_humans_annotations[0])

#########################
# MONSTERS ANNOTATIONS
#########################

monsters_annotations_path = 'pattern_ii_dataset/monsters/dataset/annotations'

all_monsters_annotations = []
    
for root, dirs, files in sorted(os.walk(monsters_annotations_path)):
    
    for name in files:
        all_monsters_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_monsters_annotations database" % len(all_monsters_annotations))

print(all_monsters_annotations[0])


loaded 36000 files into human database
loaded 6000 files into monster database
loaded 6000 files into all_humans_annotations database
pattern_ii_dataset/humans/dataset/annotations\female/subject_mesh_0001_anno.json
loaded 1000 files into all_monsters_annotations database
pattern_ii_dataset/monsters/dataset/annotations\female/subject_mesh_001_anno.json


In [3]:
##############################################################
# DATA SORTING
##############################################################

##############################
# MONSTERS
##############################

monsters_plain_paths = []
monsters_rgb_paths = []
monsters_texture_paths = []

for monster in all_monsters:
    if "rgb" in monster:
        monsters_rgb_paths.append(monster)
    elif "texture" in monster:
        monsters_texture_paths.append(monster)
    else:
        monsters_plain_paths.append(monster)

##############################
# HUMANS
##############################

humans_plain_paths = []
humans_rgb_paths = []
humans_texture_paths = []

for human in all_humans:
    if "rgb" in human:
        humans_rgb_paths.append(human)
    elif "texture" in human:
        humans_texture_paths.append(human)
    else:
        humans_plain_paths.append(human)
        
##############################
# ANNOTATIONS
##############################


monsters_annotations_df = pd.DataFrame() 

for monster_annotation_path in all_monsters_annotations:
    
    with open (monster_annotation_path, 'r') as file:
    
        # load from json
        monster_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(monster_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        monsters_annotations_df = monsters_annotations_df.append(sub_df, ignore_index=True)


humans_annotations_df = pd.DataFrame() 

for humans_annotation_path in all_humans_annotations:
    
    with open (humans_annotation_path, 'r') as file:
    
        # load from json
        human_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
    
    
#print(humans_annotations_df.head())
#print(monsters_annotations_df.head())
print(humans_rgb_paths[0])    
print(monsters_rgb_paths[0])    
print(humans_texture_paths[0])    
print(monsters_texture_paths[0])    
print(humans_plain_paths[0])    
print(monsters_plain_paths[0])

pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_rgb_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_rgb_001.png
pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_texture_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_texture_001.png
pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_001.png


In [4]:
#print(humans_annotations_df.head())
print(monsters_annotations_df.columns)

Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
       'waist_circumference'],
      dtype='object')


In [5]:
##############################################################
# CONVERT IMAGES TO GRAYSCALE
##############################################################

# reduce number of images for testing purposes
n_images = 1000000

humans_plain = []
humans_texture_grey = []
humans_rgb_grey = []
monsters_plain = []
monsters_texture_grey = []
monsters_rgb_grey = []

##############################
# HUMANS PLAIN
##############################

i = 0

for file_path in humans_plain_paths:
    human_plain = io.imread(file_path)
    
    # to vector
    humans_plain.append(human_plain.flatten())
       
    if i == n_images:
        break
        
    i += 1
    

##############################
# HUMANS W TEXTURE BACKGROUND
##############################

i = 0

for file_path in humans_texture_paths:
    human_texture_grey = color.rgb2gray(io.imread(file_path))
    
    # to vector
    human_texture_grey = human_texture_grey.flatten()
    
    humans_texture_grey.append(human_texture_grey)
       
    if i == n_images:
        break
        
    i += 1
    
##############################
# HUMANS W RGB BACKGROUND
##############################

i = 0

for file_path in humans_rgb_paths:
    human_rgb_grey = color.rgb2gray(io.imread(file_path))
    
    # to vector
    human_rgb_grey = human_rgb_grey.flatten()
    
    humans_rgb_grey.append(human_rgb_grey)
       
    if i == n_images:
        break
        
    i += 1
    
##############################
# MONSTERS PLAIN
##############################

i = 0

for file_path in monsters_plain_paths:
    monster_plain = io.imread(file_path)
    
    # to vector
    monsters_plain.append(monster_plain.flatten())
       
    if i == n_images:
        break
        
    i += 1

##############################
# MONSTERS W TEXTURE BACKGROUND
##############################

i = 0

for file_path in monsters_texture_paths:
    monster_texture_grey = color.rgb2gray(io.imread(file_path))
    
    # to vector
    monster_texture_grey = monster_texture_grey.flatten()
    
    monsters_texture_grey.append(monster_texture_grey)
       
    if i == n_images:
        break
        
    i += 1
    

##############################
# MONSTERS W RGB BACKGROUND
##############################

i = 0

for file_path in monsters_rgb_paths:
    monster_rgb_grey = color.rgb2gray(io.imread(file_path))
    
    # to vector
    monster_rgb_grey = monster_rgb_grey.flatten()
    
    monsters_rgb_grey.append(monster_rgb_grey)
       
    if i == n_images:
        break
        
    i += 1

    
#######################################################################
# DISPLAY GREY IMAGES - .flatten() has to be commented inside the loops
#######################################################################

#cv2.imshow("humans_plain", humans_plain[0])
#cv2.imshow("humans_rgb_grey", humans_rgb_grey[0])
#cv2.imshow("humans_texture_grey", humans_texture_grey[0])   
#cv2.imshow("monsters_plain", monsters_plain[0])     
#cv2.imshow("monsters_rgb_grey", monsters_rgb_grey[0])
#cv2.imshow("monsters_texture_grey", monsters_texture_grey[5])
#cv2.waitKey(0)

print("finished")

finished


In [6]:
# HIER WEITERMACHEN - STATT TENSOREN DATA FRAMES VERWENDEN <----------------------------------------------------


##############################################################
# CREATE DATA FRAME OF IMAGE VECTORS AND ANNOTATIONS
##############################################################

##############################
# HUMAN PLAIN DATA FRAME
##############################


# tensor of images w.o. annotations
humans_plain_image_tensor = torch.tensor(humans_plain)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_plain_full_tensor = torch.cat((humans_plain_image_tensor,human_annotation_tensor),1)


##############################
# HUMAN RGB DATA FRAME
##############################


# tensor of images w.o. annotations
humans_rgb_image_tensor = torch.tensor(humans_rgb_grey)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_rgb_full_tensor = torch.cat((humans_rgb_image_tensor,human_annotation_tensor),1)


##############################
# HUMAN TEXTURE DATA FRAME
##############################


# tensor of images w.o. annotations
humans_texture_image_tensor = torch.tensor(humans_texture_grey)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_texture_full_tensor = torch.cat((humans_texture_image_tensor,human_annotation_tensor),1)


##############################
# MONSTERS PLAIN DATA FRAME
##############################


# tensor of images w.o. annotations
monsters_plain_image_tensor = torch.tensor(monsters_plain)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_plain_full_tensor = torch.cat((monsters_plain_image_tensor,monster_annotation_tensor),1)


##############################
# MONSTERS RGB DATA FRAME
##############################


# tensor of images w.o. annotations
monster_rgb_image_tensor = torch.tensor(monsters_rgb_grey)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_rgb_full_tensor = torch.cat((monster_rgb_image_tensor,monster_annotation_tensor),1)


##############################
# MONSTERS TEXTURE DATA FRAME
##############################


# tensor of images w.o. annotations
monster_texture_image_tensor = torch.tensor(monsters_texture_grey)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_texture_full_tensor = torch.cat((monster_texture_image_tensor,monster_annotation_tensor),1)

print(humans_plain_full_tensor.shape)
print(humans_rgb_full_tensor.shape)
print(humans_texture_full_tensor.shape)
print(monsters_plain_full_tensor.shape)
print(monsters_rgb_full_tensor.shape)
print(monsters_texture_full_tensor.shape)




torch.Size([12000, 40008])
torch.Size([12000, 40008])
torch.Size([12000, 40008])
torch.Size([2000, 40008])
torch.Size([2000, 40008])
torch.Size([2000, 40008])


In [7]:
##############################################################
# SAVE FULL TENSORS TO CSV
##############################################################

##############################
# HUMAN AND MONSTERS PLAIN
##############################

humans_monsters_plain_full_tensor = torch.cat((humans_plain_full_tensor, monsters_plain_full_tensor),0)
#hmpf = humans_monsters_plain_full_tensor.numpy()
#hmpf_df = pd.DataFrame(hmpf)
#hmpf_df.to_csv('humans_monsters_plain.csv')


##############################
# HUMAN AND MONSTERS RGB
##############################

humans_monsters_rgb_full_tensor = torch.cat((humans_rgb_full_tensor, monsters_rgb_full_tensor),0)
#hmrgbf = humans_monsters_rgb_full_tensor.numpy()
#hmrgbf_df = pd.DataFrame(hmrgbf)
#hmrgbf_df.to_csv('humans_monsters_rgb.csv')


##############################
# HUMAN AND MONSTERS TEXTURE
##############################

humans_monsters_texture_full_tensor = torch.cat((humans_texture_full_tensor, monsters_texture_full_tensor),0)
#hmtf = humans_monsters_texture_full_tensor.numpy()
#hmtf_df = pd.DataFrame(hmtf)
#hmtf_df.to_csv('humans_monsters_texture.csv')

print("finished")

finished


In [8]:
#Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
#       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
#       'waist_circumference'],
#      dtype='object')

In [45]:
class MyData(Dataset):
    def __init__(self, dataset, train=True, transform=None):
        #Loading csv
        data_tensor=dataset
        #Leaving only image related  columns
        features=data_tensor[:,:39999]
        #print(features.shape)
        
        #Setting labels
        label_chest_circumference=data_tensor[:,40000]
        label_height=data_tensor[:,40001]
        label_inseam=data_tensor[:,40002]
        label_left_arm_length=data_tensor[:,40003]
        label_pelvis_circumference=data_tensor[:,40004]
        label_right_arm_length=data_tensor[:,40005]
        label_shoulder_width=data_tensor[:,40006]
        label_waist_circumference=data_tensor[:,40007]
        
        #Splitting the data into train and validation set
        X_train, X_test, y_chest_circumference_train, y_chest_circumference_test,\
        y_height_train, y_height_test, y_inseam_train, y_inseam_test,\
        y_left_arm_length_train, y_left_arm_length_test,\
        y_pelvis_circumference_train, y_pelvis_circumference_test,\
        y_right_arm_length_train, y_right_arm_length_test,\
        y_shoulder_width_train, y_shoulder_width_test,\
        y_waist_circumference_train, y_waist_circumference_test = train_test_split(features, label_chest_circumference, 
                           label_height, label_inseam,
                           label_left_arm_length, label_pelvis_circumference,
                           label_right_arm_length, label_shoulder_width,
                           label_waist_circumference, test_size=0.2)
        
        if train==True:
            self.x=X_train
            self.chest_circumference_y=y_chest_circumference_train
            self.height_y=y_height_train
            self.inseam_y=y_inseam_train
            self.left_arm_length_y=y_left_arm_length_train
            self.pelvis_circumference_y=y_pelvis_circumference_train
            self.right_arm_length_y=y_right_arm_length_train
            self.shoulder_width_y=y_shoulder_width_train
            self.waist_circumference_y=y_waist_circumference_train
        else:
            self.x=X_test
            self.chest_circumference_y=y_chest_circumference_test
            self.height_y=y_height_test
            self.inseam_y=y_inseam_test
            self.left_arm_length_y=y_left_arm_length_test
            self.pelvis_circumference_y=y_pelvis_circumference_test
            self.right_arm_length_y=y_right_arm_length_test
            self.shoulder_width_y=y_shoulder_width_test
            self.waist_circumference_y=y_waist_circumference_test 
        
        #Applying transformation
        self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=np.array(self.x[idx, 0:]).astype('float')
        label1=np.array([self.chest_circumference_y[idx]]).astype('float')
        label2=np.array([self.height_y[idx]]).astype('float')
        label3=np.array([self.inseam_y[idx]]).astype('float')
        label4=np.array([self.left_arm_length_y[idx]]).astype('float')
        label5=np.array([self.pelvis_circumference_y[idx]]).astype('float')
        label6=np.array([self.right_arm_length_y[idx]]).astype('float')
        label7=np.array([self.shoulder_width_y[idx]]).astype('float')
        label8=np.array([self.waist_circumference_y[idx]]).astype('float')
        
        sample={'image': np.uint8(image), 'label_chest_circumference': label1,
                'label_height': label2, 'label_inseam': label3, 'label_left_arm_length': label4,
                'label_pelvis_circumference': label5, 'label_.right_arm_length': label6, 'label_shoulder_width': label7,
                'label_waist_circumference': label8}
        
        #Applying transformation
        if self.transform:
            sample=self.transform(sample)
            
        return sample

In [48]:
#std = humans_monsters_plain_full_tensor

#normal = torchvision.transforms.Normalize(mean, std)
dataset = MyData(humans_monsters_plain_full_tensor)

In [58]:
loader = DataLoader(dataset,
                         batch_size=10,
                         num_workers=0,
                         shuffle=False)

mean = 0.0

for images in loader:
    batch_samples = len(images)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(loader.dataset)

var = 0.0
for images, _ in loader:
    batch_samples = len(images)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(loader.dataset)*224*224))

AttributeError: 'dict' object has no attribute 'view'

In [ ]:
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 8, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=1),
        
      nn.Conv2d(8, 16, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=1),
        
      nn.Flatten(),
      nn.Linear(),
        
      nn.ReLU()
    )

# 20 epochs
# mini_batc_size = 100
# Loss = MSE
# learning_rate = 0.01
# momentum = 0.9
# TODO: regressor funktion
# For layers
# 
# https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html
    
nn.
  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
  
    
    
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First 2D convolutional layer, taking in 1 input channel (image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      # Second 2D convolutional layer, taking in the 32 input layers,
      # outputting 64 convolutional features, with a square kernel size of 3
      self.conv2 = nn.Conv2d(32, 64, 3, 1)

      # Designed to ensure that adjacent pixels are either all 0s or all active
      # with an input probability
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # First fully connected layer
      self.fc1 = nn.Linear(9216, 128)
      # Second fully connected layer that outputs our 10 labels
      self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)